# 🎓 Travaux Pratiques : WGAN-GP (Wasserstein GAN with Gradient Penalty)

**Auteur :**Benlahmar Habib


## Objectifs Pédagogiques

1.  **Théorie du Wasserstein :** Comprendre le passage de la divergence Minimax (BCE) à la **Distance de Wasserstein** (ou *Earth Mover's Distance*).
2.  **Le Rôle du Critique :** Expliquer pourquoi le Discriminateur devient un **Critique** et ne prédit plus une probabilité.
3.  **Stabilité :** Implémenter la **Pénalité de Gradient (GP)** pour respecter la **Contrainte de Lipschitz** et garantir la stabilité de l'entraînement.
4.  **Boucle d'Entraînement Asymétrique :** Maîtriser le ratio d'entraînement $n_{\text{Critique}} : n_{\text{Générateur}}$.

--- 

## I. Fondements Théoriques : Wasserstein et Lipschitz

Le GAN classique (basé sur BCE) souffre de problèmes de **Gradient Vanishing** lorsque les distributions générée et réelle ne se chevauchent pas (cas fréquent). Le WGAN résout ce problème en utilisant la Distance de Wasserstein ($W$), qui fournit un gradient significatif même dans ces conditions difficiles.

### 1.1. La Distance de Wasserstein et le Critique

La perte du WGAN s'appuie sur la distance de Wasserstein (coût minimal pour transformer une distribution en une autre) :

$$\min_G W(p_{\text{data}}, p_g) \approx \min_G \max_{C \in \mathcal{C}} \mathbb{E}_{\mathbf{x} \sim p_{\text{data}}} [C(\mathbf{x})] - \mathbb{E}_{\mathbf{z} \sim p_{\mathbf{z}}} [C(G(\mathbf{z}))]$$

* **Critique ($C$) :** Mesure la distance entre les distributions. Il ne produit pas une probabilité, mais un **score** non borné (plus le score est grand pour le réel et petit pour le faux, plus la distance est grande).

### 1.2. La Contrainte de Lipschitz et la Pénalité de Gradient (GP)

Pour que $C$ soit une approximation valide de la distance $W$, il doit être une fonction **$1$-Lipschitz** : la norme de son gradient doit être inférieure ou égale à 1 partout ($\left\|\nabla_{\mathbf{x}} C(\mathbf{x})\right\| \le 1$).

Le WGAN-GP applique la Pénalité de Gradient ($GP$) pour forcer cette contrainte. Le terme est ajouté à la perte du Critique :

$$\text{Pénalité de Gradient} = \lambda \cdot \mathbb{E}_{\hat{\mathbf{x}}} [ (\| \nabla_{\hat{\mathbf{x}}} C(\hat{\mathbf{x}}) \|_2 - 1)^2 ]$$

### Question d'Accompagnement (Q1.1)

Quel est le rôle du **Critique ($C$)** dans le WGAN-GP, et en quoi diffère-t-il du rôle de classification binaire du Discriminateur dans le GAN classique ? (Indice : Que représentent les scores de sortie du Critique ?)

--- 

## II. Configuration et Architecture (DCGAN comme base)

Nous utilisons l'architecture DCGAN pour le Générateur et le Critique, en ajustant les hyperparamètres et la normalisation des données à $[-1, 1]$.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import numpy as np
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from matplotlib import pyplot as plt
from torchvision.utils import make_grid
from tqdm.notebook import trange, tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Exécution sur {device}")

# Hyperparamètres WGAN-GP
latent_dim = 100
batch_size = 128
epochs = 100
lr = 0.0001 # Taux d'apprentissage plus bas que le GAN classique
beta1 = 0.5 # Paramètre Adam
lambda_gp = 10 # Coefficient de la Pénalité de Gradient (standard = 10)
n_critic = 5 # Ratio C:G, entraîne C 5 fois pour 1 entraînement de G

# NORMALISATION : [0, 1] -> [-1, 1] pour Tanh
transform_dcgan = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize((0.5,), (0.5,)) 
])

train_dataset = datasets.FashionMNIST(root='./data/FashionMNIST', train=True, download=True, transform=transform_dcgan)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Initialisation des poids (selon les recommandations DCGAN)
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        init.normal_(m.weight.data, 1.0, 0.02)
        init.constant_(m.bias.data, 0)


### 2.1. Le Critique ($C$)

Architecture DCGAN sans `Sigmoid` ni `BatchNorm` sur la sortie, car il renvoie un score réel non borné.

In [ ]:
ndf = 64 # Nombre de 'Discriminator Feature maps'

class Critic(nn.Module):
    def __init__(self):
        super().__init__()
        self.main = nn.Sequential(
            # 1 x 28 x 28 -> ndf x 14 x 14 (Pas de BN sur la première couche)
            nn.Conv2d(1, ndf, 4, 2, 1, bias=True),
            nn.LeakyReLU(0.2, inplace=True),

            # ndf x 14 x 14 -> ndf*2 x 7 x 7
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=True), 
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),

            # ndf*2 x 7 x 7 -> ndf*4 x 4 x 4
            nn.Conv2d(ndf * 2, ndf * 4, 3, 2, 1, bias=True), 
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),

            # Sortie : ndf*4 x 4 x 4 -> 1 (score brut)
            nn.Conv2d(ndf * 4, 1, 4, 1, 0, bias=True), 
        )

    def forward(self, input):
        return self.main(input).view(-1, 1)

### 2.2. Le Générateur ($G$) (Identique au DCGAN)

ngf = 64
class Generator(nn.Module):
    def __init__(self, latent_dim):
        super().__init__()
        self.ngf = ngf
        self.main = nn.Sequential(
            # Projection initiale du bruit z (100) en volume spatial (ngf*4 x 4 x 4)
            nn.ConvTranspose2d(latent_dim, self.ngf * 4, 4, 1, 0, bias=False),
            nn.BatchNorm2d(self.ngf * 4),
            nn.ReLU(True),

            nn.ConvTranspose2d(self.ngf * 4, self.ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(self.ngf * 2),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(self.ngf * 2, self.ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(self.ngf),
            nn.ReLU(True),

            # Sortie : Tanh pour [-1, 1]
            nn.ConvTranspose2d(self.ngf, 1, 4, 2, 3, bias=False), 
            nn.Tanh()
        )

    def forward(self, z):
        return self.main(z.view(-1, latent_dim, 1, 1))

### Question d'Accompagnement (Q2.1)

Pourquoi est-il nécessaire de modifier le ratio d'entraînement pour que $n_{\text{Critique}} > n_{\text{Générateur}}$ (par exemple, 5:1), alors que le GAN classique utilisait un ratio 1:1 ? (Indice : Pensez à la stabilité et à la nécessité que le Critique soit précis pour fournir un gradient utile.)

--- 

## III. Implémentation du Gradient Penalty (GP)

Cette fonction calcule le terme de pénalité en s'assurant que la norme du gradient du Critique est proche de 1 sur les points interpolés entre les distributions réelle et fausse.

In [ ]:
def calculate_gradient_penalty(C, real_images, fake_images, lambda_gp, device):
    b_size = real_images.size(0)
    # 1. Génération des coefficients aléatoires alpha
    alpha = torch.rand(b_size, 1, 1, 1, device=device)

    # 2. Échantillonnage interpolé : x_hat = alpha * real + (1 - alpha) * fake
    x_hat = alpha * real_images + (1 - alpha) * fake_images
    x_hat.requires_grad_(True)

    # 3. Score du Critique sur les échantillons interpolés
    C_x_hat = C(x_hat)

    # 4. Calcul du gradient du Critique par rapport à x_hat
    gradients = torch.autograd.grad(outputs=C_x_hat,
                                    inputs=x_hat,
                                    grad_outputs=torch.ones_like(C_x_hat),
                                    create_graph=True,
                                    retain_graph=True)[0]

    # 5. Calcul de la norme du gradient et de la Pénalité
    gradients = gradients.view(b_size, -1)
    gradient_norm = gradients.norm(2, dim=1)

    # Pénalité : (||grad|| - 1)^2
    gp = lambda_gp * ((gradient_norm - 1)**2).mean()
    return gp

### Question d'Accompagnement (Q3.1)

Pourquoi est-il nécessaire d'échantillonner des points $\hat{\mathbf{x}}$ **entre** les distributions réelle et fausse pour calculer la Pénalité de Gradient, plutôt que de simplement calculer la pénalité sur les images réelles et fausses séparément ?

--- 

## IV. Boucle d'Entraînement WGAN-GP

La boucle d'entraînement doit respecter le ratio $n_{\text{critique}} : 1$ et intégrer la Pénalité de Gradient à la perte du Critique.

In [ ]:
# Initialisation
G = Generator(latent_dim).to(device).apply(weights_init)
C = Critic().to(device).apply(weights_init)

# Optimiseurs (Adam avec betas ajustés)
G_optimizer = optim.Adam(G.parameters(), lr=lr, betas=(beta1, 0.999))
C_optimizer = optim.Adam(C.parameters(), lr=lr, betas=(beta1, 0.999))

fixed_noise = torch.randn(64, latent_dim, device=device)

# Fonction utilitaire de visualisation
def show_grid(grid, title="", figsize=(10, 10)):
    plt.figure(figsize=figsize)
    plt.title(title)
    grid = (grid + 1) / 2 # Inverse la normalisation pour l'affichage : [-1, 1] -> [0, 1]
    plt.imshow(np.transpose(grid.numpy(), (1, 2, 0)), cmap="gray")
    plt.axis("off")
    plt.show()

def train_wgangp(G, C, G_optimizer, C_optimizer, dataloader, epochs, latent_dim, device, lambda_gp, n_critic):
    
    for epoch in trange(epochs, desc="Entraînement WGAN-GP"):
        for i, (real_images, _) in enumerate(dataloader):
            
            real_images = real_images.to(device)
            b_size = real_images.size(0)
            
            ############################
            # (1) Mise à jour C (Critique) : n_critic fois
            ############################
            for _ in range(n_critic):
                C_optimizer.zero_grad()
                
                # 1a. Génération d'images fausses
                noise = torch.randn(b_size, latent_dim, device=device)
                fake_images = G(noise).detach() 
                
                # 1b. Scores du Critique
                C_real = C(real_images)
                C_fake = C(fake_images)
                
                # 1c. Calcul de la Perte de Wasserstein 
                # C_real.mean() - C_fake.mean() donne la W_distance. C maximise cette valeur.
                W_distance = C_real.mean() - C_fake.mean()
                C_W_loss = -W_distance # Pour la minimisation avec l'optimiseur (Loss = -W_distance)
                
                # 1d. Calcul de la Pénalité de Gradient (GP)
                gp = calculate_gradient_penalty(C, real_images, fake_images, lambda_gp, device)
                
                # 1e. Perte totale du Critique
                C_loss = C_W_loss + gp
                
                C_loss.backward()
                C_optimizer.step()
                
            ############################
            # (2) Mise à jour G (Générateur) : 1 fois
            ############################
            G.zero_grad()
            
            # 2a. Score du Critique sur les fausses images
            noise = torch.randn(b_size, latent_dim, device=device)
            fake_images = G(noise)
            C_fake = C(fake_images)
            
            # 2b. Perte de G : G tente de MAXIMISER le score des fausses images
            G_loss = -C_fake.mean() # Minimiser -Score(Faux) = Maximiser Score(Faux)
            
            G_loss.backward()
            G_optimizer.step()
            
        # Affichage de l'évolution (par époque)
        tqdm.write(f"Epoch {epoch+1:2d} | C Loss: {C_loss.item():.4f} | W Dist: {W_distance.item():.4f} | G Loss: {G_loss.item():.4f}")
        
        # 3. Visualisation
        if (epoch + 1) % 5 == 0:
            G.eval()
            with torch.no_grad():
                generated_images = G(fixed_noise).cpu()
                show_grid(make_grid(generated_images, 8), title=f"WGAN-GP Génération Époque {epoch+1}")
            G.train()

# train_wgangp(G, C, G_optimizer, C_optimizer, train_dataloader, epochs, latent_dim, device, lambda_gp, n_critic) # <-- DÉCOMMENTER POUR LANCER L'ENTRAINEMENT

--- 

## V. Synthèse et Ouverture (Post-Entraînement)

### Questions Finales

1.  **Interprétabilité de la Métrique :** Que représente la valeur `W Dist` affichée à la fin de chaque époque, et pourquoi est-elle considérée comme une **meilleure métrique** de convergence et de qualité que la BCE Loss des GANs classiques ?
2.  **Rôle de $\lambda_{GP}$ :** Le coefficient de pénalité $\lambda_{GP}$ est fixé à 10. Que se passerait-il si vous fixiez $\lambda_{GP}$ à une valeur très élevée (par exemple 100) ? Quel serait l'impact sur les scores du Critique et sur la rapidité de convergence du Générateur ?
3.  **Comparaison de Stabilité :** Décrivez comment la boucle d'entraînement du WGAN-GP (avec $n_{\text{critique}} > 1$ et GP) réduit les risques de **Mode Collapse** par rapport au DCGAN classique.
4.  **Prochaines Étapes :** Le WGAN-GP a grandement amélioré la stabilité des GANs. Pour aller plus loin dans la qualité et la **manipulation sémantique** des images, quel type d'architecture moderne (par exemple, **StyleGAN**) intègre le WGAN-GP tout en se concentrant sur la manipulation du bruit latent à différentes échelles de l'image ?